## Analyzing CIA Factbook Data

### Introduction

In the following project, I will work with [CIA World Factbook](https://www.cia.gov/the-world-factbook/countries/antarctica/) dataset to analyze the data using SQL.

The Factbook dataset contains the following columns:

- name - the name of the country
- area - the country's total area (both land and area)
- area_land — the country's land area in square kilometers.
- area_water — the country's waterarea in square kilometers.
- population — the country's population.
- population_growth— the country's population growth as a percentage.
- birth_rate — the country's birth rate, or the number of births per year per 1,000 people.
- death_rate — the country's death rate, or the number of death per year per 1,000 people.



In [1]:
# Connect the notebook

In [2]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

### Overview of the Data

In [3]:
%%sql
SELECT *
    FROM facts
    LIMIT 10;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46
6,ac,Antigua and Barbuda,442,442,0,92436,1.24,15.85,5.69,2.21
7,ar,Argentina,2780400,2736690,43710,43431886,0.93,16.64,7.33,0.0
8,am,Armenia,29743,28203,1540,3056382,0.15,13.61,9.34,5.8
9,as,Australia,7741220,7682300,58920,22751014,1.07,12.15,7.14,5.65
10,au,Austria,83871,82445,1426,8665550,0.55,9.41,9.42,5.56


### Summary Statistics

In [4]:
%%sql
SELECT MIN(population) AS min_pop, MAX(population) AS max_pop, MIN(population_growth) AS min_pop_growth, MAX(population_growth) AS max_pop_growth
    FROM facts;

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


### Exploring Outliers

In [5]:
%%sql
SELECT *
    FROM facts
    WHERE population == (SELECT MIN(population)
                            FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [6]:
%%sql
SELECT *
    FROM facts
    WHERE population == (SELECT MAX(population)
                            FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


### Summary Statistics revisited

In [7]:
%%sql
SELECT MIN(population) AS min_pop, MAX(population) AS max_pop, MIN(population_growth) AS min_pop_growth, MAX(population_growth) AS max_pop_growth
    FROM facts
    WHERE name != 'World';

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


### Exploring Average Population and Area

In [8]:
%%sql
SELECT AVG(population) AS avg_pop, AVG(area) AS avg_area
    FROM facts;

 * sqlite:///factbook.db
Done.


avg_pop,avg_area
62094928.32231405,555093.546184739


In [9]:
%%sql
SELECT *
    FROM facts
    WHERE population > (SELECT AVG(population)
                            FROM facts
                           WHERE name <> 'World')
    AND area < (SELECT AVG(area)
                   FROM facts
                   WHERE name <> 'World');

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


### Most Populated Country. Highest Growth Rate Country.

In [10]:
%%sql
SELECT name AS most_populated
    FROM facts
    WHERE population == (SELECT MAX(population)
                            FROM facts
                            WHERE name <> 'World');

 * sqlite:///factbook.db
Done.


most_populated
China


In [11]:
%%sql
SELECT population_growth AS highest_growth
    FROM facts
    WHERE population_growth == (SELECT MAX(population_growth)
                                    FROM facts);

 * sqlite:///factbook.db
Done.


highest_growth
4.02


In [12]:
%%sql
SELECT *
    FROM facts
    WHERE population == (SELECT MAX(population)
                            FROM facts
                            WHERE name <> 'World') 
    OR population_growth == (SELECT MAX(population_growth)
                                  FROM facts
                                  WHERE name <> 'World');

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47


### Countries with Highest ratios of water to land. Countries with more water than land

In [13]:
%%sql
SELECT name
    FROM facts
    WHERE area_water / area_land;

 * sqlite:///factbook.db
Done.


name
British Indian Ocean Territory
Virgin Islands


In [14]:
%%sql
SELECT name
    FROM facts
    WHERE area_water > area_land
    LIMIT 5;

 * sqlite:///factbook.db
Done.


name
British Indian Ocean Territory
Virgin Islands


In [15]:
%%sql
SELECT name, area
    FROM facts
    WHERE area_water / area_land
    AND area_water > area_land;

 * sqlite:///factbook.db
Done.


name,area
British Indian Ocean Territory,54400
Virgin Islands,1910


### Countries will add most people next year.

In [16]:
%%sql
SELECT name  AS projected_population
    FROM facts
    WHERE population * population_growth
    AND name != 'World'
    ORDER BY projected_population DESC
    LIMIT 5;

 * sqlite:///factbook.db
Done.


projected_population
Zimbabwe
Zambia
Yemen
Western Sahara
West Bank


### Countries with a higher death rate than birth rate

In [17]:
%%sql
SELECT name, death_rate, birth_rate
    FROM facts
    WHERE death_rate > birth_rate
    ORDER BY name DESC
    LIMIT 5;

 * sqlite:///factbook.db
Done.


name,death_rate,birth_rate
Ukraine,14.46,10.72
Slovenia,11.37,8.42
Serbia,13.66,9.08
Saint Pierre and Miquelon,9.72,7.42
Russia,13.69,11.6


In [18]:
%%sql
SELECT name, death_rate, birth_rate
    FROM facts
    WHERE death_rate > birth_rate
    ORDER BY death_rate > birth_rate DESC
    LIMIT 5;

 * sqlite:///factbook.db
Done.


name,death_rate,birth_rate
Austria,9.42,9.41
Belarus,13.36,10.7
Bosnia and Herzegovina,9.75,8.87
Bulgaria,14.44,8.92
Croatia,12.18,9.45


### Countries with a higher birth rate than death rate

In [19]:
%%sql
SELECT name, birth_rate, death_rate
    FROM facts
    WHERE birth_rate > death_rate
    ORDER BY birth_rate / death_rate DESC
    LIMIT 10;

 * sqlite:///factbook.db
Done.


name,birth_rate,death_rate
Gaza Strip,31.11,3.04
Kuwait,19.91,2.18
Iraq,31.45,3.77
United Arab Emirates,15.43,1.97
Oman,24.44,3.36
Jordan,25.37,3.79
Solomon Islands,25.77,3.85
West Bank,22.99,3.5
Qatar,9.84,1.53
Vanuatu,25.04,4.09


### Countries with Highest population / area

In [20]:
%%sql
SELECT name, population, area
    FROM facts
    WHERE population / area
    ORDER BY population / area DESC
    LIMIT 12;

 * sqlite:///factbook.db
Done.


name,population,area
Macau,592731,28
Monaco,30535,2
Singapore,5674472,697
Hong Kong,7141106,1108
Gaza Strip,1869055,360
Gibraltar,29258,6
Bahrain,1346613,760
Maldives,393253,298
Malta,413965,316
Bermuda,70196,54
